In [2]:
import pandas as pd
from pathlib import Path
from dateutil.tz import gettz
from datetime import timezone
from dateutil.parser import parse

In [3]:
def extract_metadata(metadata_path: Path):
    with open(metadata_path) as f:
        metadata = {}
        for line in f:
            raw = line.strip().removeprefix("# ").split(": ", maxsplit=1)
            if len(raw) != 2:
                continue
            k, v = raw
            if k in ("Latitude", "Longitude"):
                k = k.lower()
                v = float(v)
                metadata[k] = v if v > 0 else pd.NA
            elif k == "Location identifier":
                country_code, station_code, station_name = v.split("_", maxsplit=2)
                assert country_code == "VN"
                metadata["country_code"] = country_code
                metadata["station_code"] = int(station_code)
                metadata["station_name"] = station_name.strip("[]")
            elif k.startswith("Value "):
                metadata[
                    k.removeprefix("Value ").lower() if k != "Value units" else "unit"
                ] = v
            elif k == "UTC offset":
                name = v.strip("()")
                metadata["tz"] = timezone(gettz(v.strip("()")).utcoffset(0), name=name)
        return metadata


In [4]:
dfs = []
stations = []

for data_path in Path("data").glob("*.csv"):
    df_out = pd.DataFrame()
    name_prefix = data_path.with_suffix("").name
    metadata_path = (
        data_path.parent / ("{}__metadata.txt").format(name_prefix)
    ).resolve(True)

    metadata = extract_metadata(metadata_path)
    df = pd.read_csv(data_path)

    for k in ("Country Code", "Station Code", "Parameter", "Label", "Unit"):
        k_out = k.lower().replace(" ", "_")
        assert (df[k] == metadata[k_out]).all()
        if k in ("Station Code", "Parameter"):
            df_out[k_out] = df[k]
    df_out["value"] = df["Value"]

    tz = metadata.pop("tz")
    ts_col = "Timestamp ({})".format(tz.tzname(None))
    df_out["timestamp"] = df[ts_col].apply(lambda x: parse(x).replace(tzinfo=tz))

    dfs.append(df_out)
    stations.append(metadata)

In [5]:
df = pd.concat(dfs, ignore_index=True)
station_df = pd.DataFrame(stations)


In [6]:
df.to_csv("data.csv")
station_df.to_csv("station.csv")


In [7]:
df

,station_code,parameter,value,timestamp
0,988315,Water Temperature,28.0,2003-01-14 17:00:00+07:00
1,988315,Water Temperature,28.5,2003-01-15 05:00:00+07:00
2,988315,Water Temperature,29.0,2003-02-14 17:00:00+07:00
3,988315,Water Temperature,29.0,2003-02-15 05:00:00+07:00
4,988315,Water Temperature,28.5,2003-03-14 17:00:00+07:00
...,...,...,...,...
1362130,440601,Conductivity,4.4,1994-11-14 17:00:00+07:00
1362131,440601,Conductivity,5.2,1994-12-14 17:00:00+07:00
1362132,440601,Conductivity,4.2,1995-01-14 17:00:00+07:00
1362133,440601,Conductivity,4.9,1995-02-14 17:00:00+07:00


In [8]:
df_wide = df.pivot_table(index=["station_code", "timestamp"], columns=["parameter"], values=["value"]).reset_index()


In [9]:
print(len(station_df["station_code"].unique()))

107

In [10]:
df_wide

station_code                 timestamp             value  \
parameter                                        Bentho AAbundance   
0                19803 1969-04-07 00:00:00+07:00               NaN   
1                19803 1969-04-08 00:00:00+07:00               NaN   
2                19803 1969-04-09 00:00:00+07:00               NaN   
3                19803 1969-04-10 00:00:00+07:00               NaN   
4                19803 1969-04-11 00:00:00+07:00               NaN   
...                ...                       ...               ...   
1324496         988316 2021-08-14 17:00:00+07:00               NaN   
1324497         988316 2021-09-14 17:00:00+07:00               NaN   
1324498         988316 2021-10-14 17:00:00+07:00               NaN   
1324499         988316 2021-11-14 17:00:00+07:00               NaN   
1324500         988316 2021-12-14 17:00:00+07:00               NaN   

                                                                       \
parameter Conductivity Diatoms AAbundance Dissolved Oxygen Fish Count   
0                  NaN                NaN              NaN        NaN   
1                  NaN                NaN              NaN        NaN   
2                  NaN                NaN              NaN        NaN   
3                  NaN                NaN              NaN        NaN   
4                  NaN                NaN              NaN        NaN   
...                ...                ...              ...        ...   
1324496          21.75                NaN             5.43        NaN   
1324497          20.40                NaN             5.09        NaN   
1324498          13.16                NaN             5.43        NaN   
1324499          18.63                NaN             5.43        NaN   
1324500          13.15                NaN             5.43        NaN   

                                                                      \
parameter Littoral AAbundance Rainfall Water Level Water Temperature   
0                         NaN      NaN        1.22               NaN   
1                         NaN      NaN        1.26               NaN   
2                         NaN      NaN        1.26               NaN   
3                         NaN      NaN        1.21               NaN   
4                         NaN      NaN        1.10               NaN   
...                       ...      ...         ...               ...   
1324496                   NaN      NaN         NaN              28.6   
1324497                   NaN      NaN         NaN              28.1   
1324498                   NaN      NaN         NaN              28.7   
1324499                   NaN      NaN         NaN              29.6   
1324500                   NaN      NaN         NaN              29.2   

                                         
parameter Zooplanktons AAbundance    pH  
0                             NaN   NaN  
1                             NaN   NaN  
2                             NaN   NaN  
3                             NaN   NaN  
4                             NaN   NaN  
...                           ...   ...  
1324496                       NaN  7.07  
1324497                       NaN  7.64  
1324498                       NaN  7.24  
1324499                       NaN  7.29  
1324500                       NaN  6.86  

[1324501 rows x 13 columns]

In [11]:
df.to_csv("parameter-long.csv", index = False)

In [12]:
df_wide.to_csv("parameter-wide.csv", index = False)

In [13]:
station_df.to_csv("station.csv", index = False)